
# Finance & Analytics Club

## Renko Chart

It is built using price movement and not price against standardised time intervals - this filters out the noise and lets you visualize the true trend.

Price movements are represented as bricks stacked at 45 degrees to each other. A new brick is added to the chart only when the price moves by a pre determined amount in either direction.

Renko Charts have a time axis, but the time scale is not fixed. Some bricks may take longer to form than other, depending on the time it takes the price to move the required brick size.

Renko charts typically use only closing price based on the chart time framne choosen.

More at: https://www.investopedia.com/terms/r/renkochart.asp

Import necessary libraries

In [1]:
import pandas_datareader.data as pdr
import datetime
from stocktrends import Renko #go through the documentation page and download stocktrends https://github.com/ChillarAnand/stocktrends

Download historical data for required stocks

In [3]:
ticker = "AAPL"
ohlcv = pdr.get_data_yahoo(ticker,datetime.date.today()-datetime.timedelta(364),datetime.date.today())
ohlcv

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-04-01,191.679993,188.380005,191.639999,191.240005,27862000.0,188.769211
2019-04-02,194.460007,191.050003,191.089996,194.020004,22765700.0,191.513290
2019-04-03,196.500000,193.149994,193.250000,195.350006,23271800.0,192.826111
2019-04-04,196.369995,193.139999,194.789993,195.690002,19114300.0,193.161682
2019-04-05,197.100006,195.929993,196.449997,197.000000,18526600.0,194.454758
...,...,...,...,...,...,...
2020-03-24,247.690002,234.300003,236.360001,246.880005,71882800.0,246.880005
2020-03-25,258.250000,244.300003,250.750000,245.520004,75900500.0,245.520004
2020-03-26,258.679993,246.360001,246.520004,258.440002,63021800.0,258.440002


### Function to calculate True Range and Average True Range

In [4]:
def ATR(DF,n):
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

In [9]:
df2 = ATR(ohlcv, 14)
df2 = df2.dropna()
df2

,High,Low,Open,Close,Volume,Adj Close,TR,ATR
Date,,,,,,,,
2019-04-22,204.940002,202.339996,202.830002,204.529999,19439500.0,201.887497,3.713852,4.451127
2019-04-23,207.750000,203.899994,204.429993,207.479996,23323000.0,204.799362,5.862503,4.463392
2019-04-24,208.479996,207.050003,207.360001,207.160004,17540600.0,204.483505,3.680634,4.370101
2019-04-25,207.759995,205.119995,206.830002,205.279999,18543200.0,202.627808,3.276489,4.351001
2019-04-26,205.000000,202.119995,204.899994,204.300003,18649100.0,201.660461,2.880005,4.275407
...,...,...,...,...,...,...,...,...
2020-03-24,247.690002,234.300003,236.360001,246.880005,71882800.0,246.880005,23.320007,20.632143
2020-03-25,258.250000,244.300003,250.750000,245.520004,75900500.0,245.520004,13.949997,20.819287
2020-03-26,258.679993,246.360001,246.520004,258.440002,63021800.0,258.440002,13.159988,20.924286


### Function to convert ohlc data into renko bricks

In [10]:
def renko_DF(DF):
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,5,6]]
    df.rename(columns = {"Date" : "date", "High" : "high","Low" : "low", "Open" : "open","Adj Close" : "close", "Volume" : "volume"}, 
              inplace = True) # we have to change the column names as mentioned in the documentation in order to make it work!
    df2 = Renko(df)
    df2.brick_size = round(ATR(DF,120)["ATR"][-1],0)
    renko_df = df2.get_ohlc_data() 
    return renko_df

In [11]:
df = renko_DF(df2)
df = df.dropna()
df

,date,open,high,low,close,uptrend
0,2019-04-22,192.0,200.0,192.0,200.0,True
1,2019-05-03,200.0,208.0,200.0,208.0,True
2,2019-05-13,200.0,200.0,192.0,192.0,False
3,2019-05-20,192.0,192.0,184.0,184.0,False
4,2019-05-29,184.0,184.0,176.0,176.0,False
5,2019-06-11,184.0,192.0,184.0,192.0,True
6,2019-07-02,192.0,200.0,192.0,200.0,True
7,2019-07-31,200.0,208.0,200.0,208.0,True
8,2019-08-05,200.0,200.0,192.0,192.0,False
9,2019-08-19,200.0,208.0,200.0,208.0,True
